In [ ]:
import random
import pandas as pd
import seaborn as sns

from eth_utils import humanize_hash, to_bytes
from clr import *

In [ ]:
match_budget = 6000

# generate some sample contributions to play with

recipient_ids = ['pizza', 'bonding_curve', 'tcr', 'pool_together', 'unisocks_fork']
funder_ids = ['0x' + str(i) for i in range(10000000, 10000020)]

recipients = random.choices(recipient_ids, k = 40)
funders = random.choices(funder_ids, k = 40)
contributions = [random.randint(1, 10) * 10**18 for i in range(40)]


In [ ]:
grants, lr_matches, clr_matches = clr(recipients, funders, contributions, match_budget)

In [ ]:
# create tidy dataframe with all data
# columns: recipient, funder, contribution, funder_total, recipient_grant, LR_match, CLR_match
# rows: unique observations

df = pd.DataFrame.from_dict(grants) \
    .rename_axis('funder').reset_index() \
    .melt(id_vars = ['funder'], value_name = 'contribution') \
    .rename(columns = {'variable':'recipient'}) \
    .dropna().reset_index(drop = True) \
    .assign(short_funder = lambda d: d['funder'].map(lambda funder: humanize_hash(to_bytes(hexstr = funder))),
            funder_total_contribution = lambda d: d.groupby('funder')['contribution'].transform('sum'),
            recipient_grant = lambda d: d.groupby('recipient')['contribution'].transform('sum'),
            LR_match = lambda d: d['recipient'].map(lambda recipient: lr_matches[recipient]),
            CLR_match = lambda d: d['recipient'].map(lambda recipient: clr_matches[recipient]),
            total_grant = df.recipient_grant + df.CLR_match
           )


# df2 = df.groupby('funder').sum()
# df2

df.head()

In [ ]:
# setup plotting
%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
by_recip = df.groupby('recipient').max().drop(['funder',
                                                   'short_funder', 
                                                   'funder_total_contribution',
                                                   'contribution'], axis = 1)
by_recip['contributions'] = df.groupby('recipient')['contribution'].count()
by_recip['mean_contrib'] = df.groupby('recipient')['contribution'].mean()
by_recip = by_recip.sort_values(by = 'total_grant', ascending = False)

by_recip

In [ ]:
# plot grant sums by number of funders by recipient
sns.relplot(x = 'contributions', y = 'CLR_match', size = 'mean_contrib', data = by_recip) ;

In [ ]:
# plot clr matches + grant sums by recipient
sns.relplot(x = 'recipient_grant', y = 'CLR_match', size = 'contributions', data = by_recip) ;

In [ ]:
# plot contributions
sns.swarmplot(x = 'recipient', y = 'contribution', data = df) ;